1. How to import PySpark and check the version?

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark=SparkSession.builder.master("local[1]").appName("SparkAssessment.com").getOrCreate()

print(spark.version)

24/09/25 11:45:03 WARN Utils: Your hostname, AI-CJB-LAP-459 resolves to a loopback address: 127.0.1.1; using 192.168.1.164 instead (on interface wlp0s20f3)
24/09/25 11:45:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/25 11:45:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


3.5.2


In [2]:
from pyspark.sql.functions import monotonically_increasing_id,row_number,min,max
from pyspark.sql import Window,functions as F

2. How to convert the index of a PySpark DataFrame into a column?


In [3]:
df = spark.createDataFrame([
("Alice", 1),
("Bob", 2),
("Charlie", 3),
], ["Name", "Value"])

df.show()
df_with_index=df.withColumn("index",monotonically_increasing_id())
df_with_index.show()

#or
window_spec=Window.orderBy("value")
df_index=df.withColumn("index",row_number().over(window_spec)-1)
df_index.show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+

+-------+-----+-----+
|   Name|Value|index|
+-------+-----+-----+
|  Alice|    1|    0|
|    Bob|    2|    1|
|Charlie|    3|    2|
+-------+-----+-----+



24/09/25 11:45:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+-----+-----+
|   Name|Value|index|
+-------+-----+-----+
|  Alice|    1|    0|
|    Bob|    2|    1|
|Charlie|    3|    2|
+-------+-----+-----+



3. How to combine many lists to form a PySpark DataFrame?

In [4]:
list1 = ["a", "b", "c", "d"]
list2 = [1, 2, 3, 4]
data=list(zip(list1,list2))
df=spark.createDataFrame(data,["c1","c2"])
df.show()

+---+---+
| c1| c2|
+---+---+
|  a|  1|
|  b|  2|
|  c|  3|
|  d|  4|
+---+---+



4. How to get the items of list A not present in list B?

In [5]:
list_A = [1, 2, 3, 4, 5]
list_B = [4, 5, 6, 7, 8]

rdd1=spark.sparkContext.parallelize(list_A)
rdd2=spark.sparkContext.parallelize(list_B)

res=rdd1.subtract(rdd2)
res.collect()

[2, 1, 3]

5.How to get the items not common to both list A and list B?

In [6]:
res1=rdd2.subtract(rdd1)
res_union=res.union(res1)
res_union.collect()

[2, 1, 3, 6, 8, 7]

6. How to get the minimum, 25th percentile, median, 75th, and max of a numeric column?

In [7]:
data = [("A", 10), ("B", 20), ("C", 30), ("D", 40), ("E", 50), ("F", 15), ("G", 28), ("H", 54), ("I", 41), ("J", 86)]
df = spark.createDataFrame(data, ["Name", "Age"])

df.show()
min_val=df.agg(min("age")).collect()[0][0]
max_val=df.agg(max("age")).collect()[0][0]

quantiles=df.approxQuantile("age",[0.25,0.50,0.75],0.01)

print("min value:",min_val)
print("25th quartile:",quantiles[0])
print("median:",quantiles[1])
print("75th quartile:",quantiles[2])
print("max val",max_val)

+----+---+
|Name|Age|
+----+---+
|   A| 10|
|   B| 20|
|   C| 30|
|   D| 40|
|   E| 50|
|   F| 15|
|   G| 28|
|   H| 54|
|   I| 41|
|   J| 86|
+----+---+

min value: 10
25th quartile: 20.0
median: 30.0
75th quartile: 50.0
max val 86


7. How to get frequency counts of unique items of a column?

In [8]:
from pyspark.sql import Row

# Sample data
data = [
Row(name='John', job='Engineer'),
Row(name='John', job='Engineer'),
Row(name='Mary', job='Scientist'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Scientist'),
Row(name='Sam', job='Doctor'),
]

# create DataFrame
df = spark.createDataFrame(data)

# show DataFrame
df.show()
name_count=df.groupBy("job").count()
name_count.show()


+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|   Doctor|
+----+---------+

+---------+-----+
|      job|count|
+---------+-----+
|Scientist|    2|
|   Doctor|    1|
| Engineer|    4|
+---------+-----+



8. How to keep only top 2 most frequent values as it is and replace everything else as ‘Other’?

In [9]:
top_2_jobs = name_count.orderBy(F.desc("count")).limit(2).select("job").rdd.flatMap(lambda x: x).collect()

# Replace other jobs with 'Other'
result_df = df.withColumn("job", F.when(df["job"].isin(top_2_jobs), df["job"]).otherwise("Other"))

# Show the resulting DataFrame
result_df.show()

+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|    Other|
+----+---------+



9. How to Drop rows with NA values specific to a particular column?

In [10]:
# Assuming df is your DataFrame
df = spark.createDataFrame([
("A", 1, None),
("B", None, "123" ),
("B", 3, "456"),
("D", None, None),
], ["Name", "Value", "id"])

df.show()

+----+-----+----+
|Name|Value|  id|
+----+-----+----+
|   A|    1|NULL|
|   B| NULL| 123|
|   B|    3| 456|
|   D| NULL|NULL|
+----+-----+----+



In [11]:
df.dropna(subset="id").show()

+----+-----+---+
|Name|Value| id|
+----+-----+---+
|   B| NULL|123|
|   B|    3|456|
+----+-----+---+



10. How to rename columns of a PySpark DataFrame using two lists – one containing the old column names and the other containing the new column names?

In [12]:

df = spark.createDataFrame([(1, 2, 3), (4, 5, 6)], ["col1", "col2", "col3"])

old_names = ["col1", "col2", "col3"]

new_names = ["new_col1", "new_col2", "new_col3"]

df.show()

for old_name, new_name in zip(old_names, new_names):
    df = df.withColumnRenamed(old_name, new_name)
print("DataFrame with Renamed Columns:")
df.show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|   2|   3|
|   4|   5|   6|
+----+----+----+

DataFrame with Renamed Columns:
+--------+--------+--------+
|new_col1|new_col2|new_col3|
+--------+--------+--------+
|       1|       2|       3|
|       4|       5|       6|
+--------+--------+--------+



11. How to bin a numeric list to 10 groups of equal size?
CONFUSING.

In [13]:
from pyspark.sql.functions import rand,initcap
from pyspark.ml.feature import Bucketizer

# Create a DataFrame with a single column "values" filled with random numbers
num_items = 100
df = spark.range(num_items).select(rand(seed=42).alias("values"))
bucketizer=Bucketizer(splits=[0,10,float("Inf")],inputCol="values",outputCol="Buckets")
df_buck=bucketizer.setHandleInvalid("keep").transform(df)
df_buck.show(5)
df.show(5)

+-------------------+-------+
|             values|Buckets|
+-------------------+-------+
|  0.619189370225301|    0.0|
| 0.5096018842446481|    0.0|
| 0.8325259388871524|    0.0|
|0.26322809041172357|    0.0|
| 0.6702867696264135|    0.0|
+-------------------+-------+
only showing top 5 rows

+-------------------+
|             values|
+-------------------+
|  0.619189370225301|
| 0.5096018842446481|
| 0.8325259388871524|
|0.26322809041172357|
| 0.6702867696264135|
+-------------------+
only showing top 5 rows



12. How to create contigency table?

In [14]:
data = [("A", "X"), ("A", "Y"), ("A", "X"), ("B", "Y"), ("B", "X"), ("C", "X"), ("C", "X"), ("C", "Y")]
df = spark.createDataFrame(data, ["category1", "category2"])
df.crosstab("category1","category2").sort("category1_category2").show()
# df.show()

+-------------------+---+---+
|category1_category2|  X|  Y|
+-------------------+---+---+
|                  A|  2|  1|
|                  B|  1|  1|
|                  C|  2|  1|
+-------------------+---+---+



13. How to find the numbers that are multiples of 3 from a column?

In [15]:
from pyspark.sql.functions import rand,col

# Generate a DataFrame with a single column "id" with 10 rows
df = spark.range(10)

# Generate a random float between 0 and 1, scale and shift it to get a random integer between 1 and 10
df = df.withColumn("random", ((rand(seed=42) * 10) + 1).cast("int"))

mul_3=df.filter(col("random")%3==0)
mul_3.show()
# Show the DataFrame
# df.show()

+---+------+
| id|random|
+---+------+
|  1|     6|
|  2|     9|
|  3|     3|
|  5|     6|
+---+------+



14. How to extract items at given positions from a column?

In [16]:
from pyspark.sql.functions import rand

# Generate a DataFrame with a single column "id" with 10 rows
df = spark.range(10)

# Generate a random float between 0 and 1, scale and shift it to get a random integer between 1 and 10
df = df.withColumn("random", ((rand(seed=42) * 10) + 1).cast("int"))

# Show the DataFrame
df.show()

pos = [0, 4, 8, 5]

item=df.select("random").collect()
res=[item[i][0] for i in pos]
print(res)


+---+------+
| id|random|
+---+------+
|  0|     7|
|  1|     6|
|  2|     9|
|  3|     3|
|  4|     7|
|  5|     6|
|  6|    10|
|  7|     1|
|  8|    10|
|  9|     8|
+---+------+

[7, 7, 10, 6]


15. How to stack two DataFrames vertically ?

In [17]:
# Create DataFrame for region A
df_A = spark.createDataFrame([("apple", 3, 5), ("banana", 1, 10), ("orange", 2, 8)], ["Name", "Col_1", "Col_2"])
df_A.show()

# Create DataFrame for region B
df_B = spark.createDataFrame([("apple", 3, 5), ("banana", 1, 15), ("grape", 4, 6)], ["Name", "Col_1", "Col_3"])
df_B.show()

res_df=df_A.union(df_B)

res_df.show()

+------+-----+-----+
|  Name|Col_1|Col_2|
+------+-----+-----+
| apple|    3|    5|
|banana|    1|   10|
|orange|    2|    8|
+------+-----+-----+

+------+-----+-----+
|  Name|Col_1|Col_3|
+------+-----+-----+
| apple|    3|    5|
|banana|    1|   15|
| grape|    4|    6|
+------+-----+-----+

+------+-----+-----+
|  Name|Col_1|Col_2|
+------+-----+-----+
| apple|    3|    5|
|banana|    1|   10|
|orange|    2|    8|
| apple|    3|    5|
|banana|    1|   15|
| grape|    4|    6|
+------+-----+-----+



16. How to compute the mean squared error on a truth and predicted columns?

In [18]:
# Assume you have a DataFrame df with two columns "actual" and "predicted"
# For the sake of example, we'll create a sample DataFrame
from pyspark.sql.functions import mean, col


data = [(1, 1), (2, 4), (3, 9), (4, 16), (5, 25)]
df = spark.createDataFrame(data, ["actual", "predicted"])

df.show()
squared_error=df.withColumn("mean-squared-error",(col("actual")-col("predicted"))**2)
mse=squared_error.select(mean("mean-squared-error")).collect()[0][0]
print(mse)

+------+---------+
|actual|predicted|
+------+---------+
|     1|        1|
|     2|        4|
|     3|        9|
|     4|       16|
|     5|       25|
+------+---------+

116.8


17. How to convert the first character of each element in a series to uppercase?

In [19]:
# Suppose you have the following DataFrame
data = [("john",), ("alice",), ("bob",)]
df = spark.createDataFrame(data, ["name"])
df.withColumn("name",initcap(df["name"])).show()
df.show()


+-----+
| name|
+-----+
| John|
|Alice|
|  Bob|
+-----+

+-----+
| name|
+-----+
| john|
|alice|
|  bob|
+-----+



18. How to compute summary statistics for all columns in a dataframe

In [20]:
# For the sake of example, we'll create a sample DataFrame
data = [('James', 34, 55000),
('Michael', 30, 70000),
('Robert', 37, 60000),
('Maria', 29, 80000),
('Jen', 32, 65000)]

df = spark.createDataFrame(data, ["name", "age" , "salary"])
Describe=df.describe()
Describe.show()

+-------+------+-----------------+-----------------+
|summary|  name|              age|           salary|
+-------+------+-----------------+-----------------+
|  count|     5|                5|                5|
|   mean|  NULL|             32.4|          66000.0|
| stddev|  NULL|3.209361307176242|9617.692030835673|
|    min| James|               29|            55000|
|    max|Robert|               37|            80000|
+-------+------+-----------------+-----------------+



19. How to calculate the number of characters in each word in a column?

In [21]:
from pyspark.sql.functions import length, explode,split,lag

In [22]:
# Suppose you have the following DataFrame
data = [("john",), ("alice",), ("bob",)]
df = spark.createDataFrame(data, ["name"])

# df.show()
df.withColumn("length",length("name")).show()

+-----+------+
| name|length|
+-----+------+
| john|     4|
|alice|     5|
|  bob|     3|
+-----+------+



20 How to compute difference of differences between consecutive numbers of a column?

In [23]:
from pyspark.sql import *
from pyspark.sql.functions import *

In [24]:
# For the sake of example, we'll create a sample DataFrame
data = [('James', 34, 55000),
('Michael', 30, 70000),
('Robert', 37, 60000),
('Maria', 29, 80000),
('Jen', 32, 65000)]

df = spark.createDataFrame(data, ["name", "age" , "salary"])

df.show()
window_spec = Window.orderBy("salary")
df_with_diff = df.withColumn("salary_diff", col("salary") - lag("salary").over(window_spec))

df_with_diff_of_diff = df_with_diff.withColumn("diff_of_diff", col("salary_diff") - lag("salary_diff").over(window_spec))

df_with_diff_of_diff.select("name", "salary", "salary_diff", "diff_of_diff").show()

# window_spec=Window.orderBy("salary")
# salary_diff=df.withColumn("salary_diff",col("salary")-lag("salary").over(window_spec))
# salary_diff_diff=salary_diff.withColumn("salary_diff_diff",col("salary_diff")-lag("salary_diff").over(window_spec))
# df.select("name","age","salary","salary_diff","salary_diff_diff")


+-------+---+------+
|   name|age|salary|
+-------+---+------+
|  James| 34| 55000|
|Michael| 30| 70000|
| Robert| 37| 60000|
|  Maria| 29| 80000|
|    Jen| 32| 65000|
+-------+---+------+

+-------+------+-----------+------------+
|   name|salary|salary_diff|diff_of_diff|
+-------+------+-----------+------------+
|  James| 55000|       NULL|        NULL|
| Robert| 60000|       5000|        NULL|
|    Jen| 65000|       5000|           0|
|Michael| 70000|       5000|           0|
|  Maria| 80000|      10000|        5000|
+-------+------+-----------+------------+



24/09/25 11:45:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 1

21. How to get the day of month, week number, day of year and day of week from a date strings?

In [25]:
data = [("2023-05-18","01 Jan 2010",), ("2023-12-31", "01 Jan 2010",)]
df = spark.createDataFrame(data, ["date_str_1", "date_str_2"])

df.show()
df_with_dates = df.select(
    col("date_str_1"),
    col("date_str_2"),
    to_date(col("date_str_1")).alias("date_1"),
    to_date(col("date_str_2"), "dd MMM yyyy").alias("date_2")
)
df_final = df_with_dates.select(
    "date_str_1",
    "date_str_2",
    dayofmonth("date_1").alias("day_of_month_1"),
    weekofyear("date_1").alias("week_number_1"),
    dayofyear("date_1").alias("day_of_year_1"),
    dayofweek("date_1").alias("day_of_week_1"),
    dayofmonth("date_2").alias("day_of_month_2"),
    weekofyear("date_2").alias("week_number_2"),
    dayofyear("date_2").alias("day_of_year_2"),
    dayofweek("date_2").alias("day_of_week_2")
)
df_final.show()

+----------+-----------+
|date_str_1| date_str_2|
+----------+-----------+
|2023-05-18|01 Jan 2010|
|2023-12-31|01 Jan 2010|
+----------+-----------+

+----------+-----------+--------------+-------------+-------------+-------------+--------------+-------------+-------------+-------------+
|date_str_1| date_str_2|day_of_month_1|week_number_1|day_of_year_1|day_of_week_1|day_of_month_2|week_number_2|day_of_year_2|day_of_week_2|
+----------+-----------+--------------+-------------+-------------+-------------+--------------+-------------+-------------+-------------+
|2023-05-18|01 Jan 2010|            18|           20|          138|            5|             1|           53|            1|            6|
|2023-12-31|01 Jan 2010|            31|           52|          365|            1|             1|           53|            1|            6|
+----------+-----------+--------------+-------------+-------------+-------------+--------------+-------------+-------------+-------------+



22. How to convert year-month string to dates corresponding to the 4th day of the month?

In [26]:
# example dataframe
df = spark.createDataFrame([('Jan 2010',), ('Feb 2011',), ('Mar 2012',)], ['MonthYear'])

df.show()
df.withColumn(
    "date",
    to_date(concat(col("MonthYear"),lit(" 4")),"MMM yyyy d")).show()

+---------+
|MonthYear|
+---------+
| Jan 2010|
| Feb 2011|
| Mar 2012|
+---------+

+---------+----------+
|MonthYear|      date|
+---------+----------+
| Jan 2010|2010-01-04|
| Feb 2011|2011-02-04|
| Mar 2012|2012-03-04|
+---------+----------+



23 How to filter words that contain atleast 2 vowels from a series?

In [27]:
# example dataframe
df = spark.createDataFrame([('Apple',), ('Orange',), ('Plan',) , ('Python',) , ('Money',)], ['Word'])

df.show()
df.filter(col("word").rlike("[aeiouAEIOU]")).show()

+------+
|  Word|
+------+
| Apple|
|Orange|
|  Plan|
|Python|
| Money|
+------+

+------+
|  Word|
+------+
| Apple|
|Orange|
|  Plan|
|Python|
| Money|
+------+



24. How to filter valid emails from a list?
Difficulty Level: L3

In [28]:

# Create a list
data = ['buying books at amazom.com', 'rameses@egypt.com', 'matt@t.co', 'narendra@modi.com']

# Convert the list to DataFrame
df = spark.createDataFrame(data, "string").toDF("email")
df.show(truncate =False)
df.filter(col("email").rlike("@[a-zA-Z0-9.-]")).show()

+--------------------------+
|email                     |
+--------------------------+
|buying books at amazom.com|
|rameses@egypt.com         |
|matt@t.co                 |
|narendra@modi.com         |
+--------------------------+

+-----------------+
|            email|
+-----------------+
|rameses@egypt.com|
|        matt@t.co|
|narendra@modi.com|
+-----------------+



25. How to Pivot PySpark DataFrame?
Convert region categories to Columns and sum the revenue


In [29]:

# Sample data
data = [
(2021, 1, "US", 5000),
(2021, 1, "EU", 4000),
(2021, 2, "US", 5500),
(2021, 2, "EU", 4500),
(2021, 3, "US", 6000),
(2021, 3, "EU", 5000),
(2021, 4, "US", 7000),
(2021, 4, "EU", 6000),
]

# Create DataFrame
columns = ["year", "quarter", "region", "revenue"]
df = spark.createDataFrame(data, columns)
df.show()
df.groupBy("region").sum("revenue").show()



+----+-------+------+-------+
|year|quarter|region|revenue|
+----+-------+------+-------+
|2021|      1|    US|   5000|
|2021|      1|    EU|   4000|
|2021|      2|    US|   5500|
|2021|      2|    EU|   4500|
|2021|      3|    US|   6000|
|2021|      3|    EU|   5000|
|2021|      4|    US|   7000|
|2021|      4|    EU|   6000|
+----+-------+------+-------+

+------+------------+
|region|sum(revenue)|
+------+------------+
|    US|       23500|
|    EU|       19500|
+------+------------+



26. How to get the mean of a variable grouped by another variable?


In [30]:

# Sample data
data = [("1001", "Laptop", 1000),
("1002", "Mouse", 50),
("1003", "Laptop", 1200),
("1004", "Mouse", 30),
("1005", "Smartphone", 700)]

# Create DataFrame
columns = ["OrderID", "Product", "Price"]
df = spark.createDataFrame(data, columns)

df.show()
df.groupBy("product").mean("price").show()

+-------+----------+-----+
|OrderID|   Product|Price|
+-------+----------+-----+
|   1001|    Laptop| 1000|
|   1002|     Mouse|   50|
|   1003|    Laptop| 1200|
|   1004|     Mouse|   30|
|   1005|Smartphone|  700|
+-------+----------+-----+

+----------+----------+
|   product|avg(price)|
+----------+----------+
|    Laptop|    1100.0|
|     Mouse|      40.0|
|Smartphone|     700.0|
+----------+----------+



27. How to compute the euclidean distance between two columns?

Compute the euclidean distance between series (points) p and q, without using a packaged formula.

NEWLY LEARNED

In [31]:

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [32]:


# Define your series
data = [(1, 10), (2, 9), (3, 8), (4, 7), (5, 6), (6, 5), (7, 4), (8, 3), (9, 2), (10, 1)]

# Convert list to DataFrame
df = spark.createDataFrame(data, ["series1", "series2"])

df.show()

vecAssembler = VectorAssembler(inputCols=["series1", "series2"], outputCol="vectors")
df = vecAssembler.transform(df)

df = df.withColumn("squared_diff", expr("POW(series1 - series2, 2)"))
df.show()

df.agg(expr("SQRT(SUM(squared_diff))").alias("euclidean_distance")).show()

+-------+-------+
|series1|series2|
+-------+-------+
|      1|     10|
|      2|      9|
|      3|      8|
|      4|      7|
|      5|      6|
|      6|      5|
|      7|      4|
|      8|      3|
|      9|      2|
|     10|      1|
+-------+-------+

+-------+-------+----------+------------+
|series1|series2|   vectors|squared_diff|
+-------+-------+----------+------------+
|      1|     10|[1.0,10.0]|        81.0|
|      2|      9| [2.0,9.0]|        49.0|
|      3|      8| [3.0,8.0]|        25.0|
|      4|      7| [4.0,7.0]|         9.0|
|      5|      6| [5.0,6.0]|         1.0|
|      6|      5| [6.0,5.0]|         1.0|
|      7|      4| [7.0,4.0]|         9.0|
|      8|      3| [8.0,3.0]|        25.0|
|      9|      2| [9.0,2.0]|        49.0|
|     10|      1|[10.0,1.0]|        81.0|
+-------+-------+----------+------------+

+------------------+
|euclidean_distance|
+------------------+
| 18.16590212458495|
+------------------+



28. How to replace missing spaces in a string with the least frequent character?
NEWLY LEARNED

In [33]:
#Sample DataFrame
df = spark.createDataFrame([('dbc deb abed gade',),], ["string"])
df.show()
char_df=df.select(explode(split(col("string"),"")).alias("character"))
c_count=char_df.groupBy("character").count()
leaast_f_count=c_count.orderBy("count").first()
least_char=leaast_f_count["character"]
df.withColumn("modified_string",regexp_replace(col("string")," ",least_char)).show()
print(leaast_f_count)



+-----------------+
|           string|
+-----------------+
|dbc deb abed gade|
+-----------------+

+-----------------+-----------------+
|           string|  modified_string|
+-----------------+-----------------+
|dbc deb abed gade|dbcgdebgabedggade|
+-----------------+-----------------+

Row(character='g', count=1)


29. How to create a TimeSeries starting ‘2000-01-01’ and 10 weekends (saturdays) after that having random numbers as values?

In [34]:
start_date = '2000-01-01'
num_saturdays = 10

# Generate the list of Saturdays
saturdays = spark.range(num_saturdays).select(
    date_add(expr(f"to_date('{start_date}')"), (7 * col("id")).cast("int")).alias("date")
)
saturdays.withColumn("value",rand()).show()

+----------+-------------------+
|      date|              value|
+----------+-------------------+
|2000-01-01|0.47323951218110627|
|2000-01-08| 0.5567639307219009|
|2000-01-15| 0.8389891478755225|
|2000-01-22| 0.3027736712830308|
|2000-01-29| 0.3327693419345965|
|2000-02-05|0.38162341783725195|
|2000-02-12|0.31050425827927586|
|2000-02-19| 0.7849956118412406|
|2000-02-26| 0.7510232334631551|
|2000-03-04|0.43494979813412005|
+----------+-------------------+



30. How to get the nrows, ncolumns, datatype of a dataframe?

Get the number of rows, columns, datatype and summary statistics of each column of the Churn_Modelling dataset. Also get the numpy array and list equivalent of the dataframe

In [35]:
from pyspark import SparkFiles

In [36]:

url = "https://raw.githubusercontent.com/selva86/datasets/master/Churn_Modelling.csv"

spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("Churn_Modelling.csv"), header=True, inferSchema=True)

#df = spark.read.csv("C:/Users/RajeshVaddi/Documents/MLPlus/DataSets/Churn_Modelling.csv", header=True, inferSchema=True)

df.show(5, truncate=False)

nrows=df.count()
print("nrows:",nrows)
ncolumns=len(df.columns)
print("nColumns:",ncolumns)

data_types={col:(df.schema[col].dataType)for col in df.columns}
print("datatype:",data_types)
summary_stat=df.describe()
numpy_arr=df.toPandas().to_numpy()
print("numpy array shape: ",numpy_arr.shape)
print("summary statistics:",summary_stat)


+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|Surname |CreditScore|Geography|Gender|Age|Tenure|Balance  |NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|1        |15634602  |Hargrave|619        |France   |Female|42 |2     |0.0      |1            |1        |1             |101348.88      |1     |
|2        |15647311  |Hill    |608        |Spain    |Female|41 |1     |83807.86 |1            |0        |1             |112542.58      |0     |
|3        |15619304  |Onio    |502        |France   |Female|42 |8     |159660.8 |3            |1        |0             |113931.57      |1     |
|4        |15701354  |Boni    |699        |France   |Female|39 |1     |0.0      |2            |0        |0             |93826.63       |

31. How to rename a specific columns in a dataframe?

In [37]:
df = spark.createDataFrame([('Alice', 1, 30),('Bob', 2, 35)], ["name", "age", "qty"])

df.show()

# Rename lists for specific columns
old_names = ["qty", "age"]
new_names = ["user_qty", "user_age"]
for old_name,new_name in zip(old_names,new_names):
    
    df=df.withColumnRenamed(old_name,new_name)
df.show()


+-----+---+---+
| name|age|qty|
+-----+---+---+
|Alice|  1| 30|
|  Bob|  2| 35|
+-----+---+---+

+-----+--------+--------+
| name|user_age|user_qty|
+-----+--------+--------+
|Alice|       1|      30|
|  Bob|       2|      35|
+-----+--------+--------+



32. How to check if a dataframe has any missing values and count of missing values in each column?

In [38]:
from pyspark.sql import functions as F

In [39]:
# Assuming df is your DataFrame
df = spark.createDataFrame([
("A", 1, None),
("B", None, "123" ),
("B", 3, "456"),
("D", None, None),
], ["Name", "Value", "id"])

df.show()
null_count=df.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_count.show()


+----+-----+----+
|Name|Value|  id|
+----+-----+----+
|   A|    1|NULL|
|   B| NULL| 123|
|   B|    3| 456|
|   D| NULL|NULL|
+----+-----+----+

+----+-----+---+
|Name|Value| id|
+----+-----+---+
|   0|    2|  2|
+----+-----+---+



33 How to replace missing values of multiple numeric columns with the mean?

MAKE USE OF IMPUTER FUNCTION RATHER USING FILL OR FILLNA

In [40]:
from pyspark.ml.feature import Imputer

In [41]:
df = spark.createDataFrame([
("A", 1, None),
("B", None, 123 ),
("B", 3, 456),
("D", 6, None),
], ["Name", "var1", "var2"])

df.show()
cols=["var1","var2"]
imputer=Imputer(inputCols=cols,outputCols=cols,strategy="mean")
model=imputer.fit(df)
imputed_df=model.transform(df)
imputed_df.show()

+----+----+----+
|Name|var1|var2|
+----+----+----+
|   A|   1|NULL|
|   B|NULL| 123|
|   B|   3| 456|
|   D|   6|NULL|
+----+----+----+

+----+----+----+
|Name|var1|var2|
+----+----+----+
|   A|   1| 289|
|   B|   3| 123|
|   B|   3| 456|
|   D|   6| 289|
+----+----+----+



34. How to change the order of columns of a dataframe?

In [42]:
# Sample data
data = [("John", "Doe", 30), ("Jane", "Doe", 25), ("Alice", "Smith", 22)]

# Create DataFrame from the data
df = spark.createDataFrame(data, ["First_Name", "Last_Name", "Age"])

# Show the DataFrame
df.show()
new_order=["Age","First_Name","Last_Name"]
df.select(*new_order).show()

+----------+---------+---+
|First_Name|Last_Name|Age|
+----------+---------+---+
|      John|      Doe| 30|
|      Jane|      Doe| 25|
|     Alice|    Smith| 22|
+----------+---------+---+

+---+----------+---------+
|Age|First_Name|Last_Name|
+---+----------+---------+
| 30|      John|      Doe|
| 25|      Jane|      Doe|
| 22|     Alice|    Smith|
+---+----------+---------+



35. How to format or suppress scientific notations in a PySpark DataFrame?


In [43]:
# Assuming you have a DataFrame df and the column you want to format is 'your_column'
df = spark.createDataFrame([(1, 0.000000123), (2, 0.000023456), (3, 0.000345678)], ["id", "your_column"])

df.show()
df.withColumn("your_column",format_number("your_column",10)).show()

+---+-----------+
| id|your_column|
+---+-----------+
|  1|    1.23E-7|
|  2|  2.3456E-5|
|  3| 3.45678E-4|
+---+-----------+

+---+------------+
| id| your_column|
+---+------------+
|  1|0.0000001230|
|  2|0.0000234560|
|  3|0.0003456780|
+---+------------+



36. How to format all the values in a dataframe as percentages?

In [44]:
# Sample data
data = [(0.1, .08), (0.2, .06), (0.33, .02)]
df = spark.createDataFrame(data, ["numbers_1", "numbers_2"])

df.show()
# df.withColumn("numbers_2",format_number("numbers_2",100)).show()
df_percentage = df.select(
    [F.concat(F.format_number(F.col(col_name) * 100, 2), F.lit('%')).alias(col_name) for col_name in df.columns]
)
df_percentage.show()

+---------+---------+
|numbers_1|numbers_2|
+---------+---------+
|      0.1|     0.08|
|      0.2|     0.06|
|     0.33|     0.02|
+---------+---------+

+---------+---------+
|numbers_1|numbers_2|
+---------+---------+
|   10.00%|    8.00%|
|   20.00%|    6.00%|
|   33.00%|    2.00%|
+---------+---------+



37. How to filter every nth row in a dataframe?

In [45]:
# Sample data
data = [("Alice", 1), ("Bob", 2), ("Charlie", 3), ("Dave", 4), ("Eve", 5),
("Frank", 6), ("Grace", 7), ("Hannah", 8), ("Igor", 9), ("Jack", 10)]

# Create DataFrame
df = spark.createDataFrame(data, ["Name", "Number"])

df.show()
window_spec=Window.orderBy(monotonically_increasing_id())
df=df.withColumn("rn",row_number().over(window_spec))

n=5

df=df.filter((df.rn%n)==0)
df.show()

+-------+------+
|   Name|Number|
+-------+------+
|  Alice|     1|
|    Bob|     2|
|Charlie|     3|
|   Dave|     4|
|    Eve|     5|
|  Frank|     6|
|  Grace|     7|
| Hannah|     8|
|   Igor|     9|
|   Jack|    10|
+-------+------+

+----+------+---+
|Name|Number| rn|
+----+------+---+
| Eve|     5|  5|
|Jack|    10| 10|
+----+------+---+



24/09/25 11:45:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


38 How to get the row number of the nth largest value in a column?

In [46]:


from pyspark.sql import Row

# Sample Data
data = [
Row(id=1, column1=5),
Row(id=2, column1=8),
Row(id=3, column1=12),
Row(id=4, column1=1),
Row(id=5, column1=15),
Row(id=6, column1=7),
]

df = spark.createDataFrame(data)
df.show()

from pyspark.sql.window import Window
from pyspark.sql.functions import desc, row_number

window = Window.orderBy(desc("column1"))
df = df.withColumn("row_number", row_number().over(window))

n = 3 # We're interested in the 3rd largest value.
row = df.filter(df.row_number == n).first()

if row:
    print("Row number:", row.row_number)
print("Column value:", row.column1)


+---+-------+
| id|column1|
+---+-------+
|  1|      5|
|  2|      8|
|  3|     12|
|  4|      1|
|  5|     15|
|  6|      7|
+---+-------+

Row number: 3
Column value: 8


24/09/25 11:45:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


39. How to get the last n rows of a dataframe with row sum > 100?

In [47]:
# Sample data
data = [(10, 25, 70),
(40, 5, 20),
(70, 80, 100),
(10, 2, 60),
(40, 50, 20)]

# Create DataFrame
df = spark.createDataFrame(data, ["col1", "col2", "col3"])

# Display original DataFrame
df.show()
df_sum=df.withColumn("row_sum",F.col("col1")+F.col("col2")+F.col("col3"))
filtered_df=df_sum.filter(F.col("row_sum")>100)
n=5
last_n_rows=filtered_df.tail(n)
for row in last_n_rows:
    print(row)

+----+----+----+
|col1|col2|col3|
+----+----+----+
|  10|  25|  70|
|  40|   5|  20|
|  70|  80| 100|
|  10|   2|  60|
|  40|  50|  20|
+----+----+----+

Row(col1=10, col2=25, col3=70, row_sum=105)
Row(col1=70, col2=80, col3=100, row_sum=250)
Row(col1=40, col2=50, col3=20, row_sum=110)


40. How to create a column containing the minimum by maximum of each row?


In [48]:
from pyspark.sql.types import FloatType
from pyspark.sql import Row
from pyspark.sql.functions import udf,array

In [49]:
# Sample Data
data = [(1, 2, 3), (4, 5, 6), (7, 8, 9), (10, 11, 12)]

# Create DataFrame
df = spark.createDataFrame(data, ["col1", "col2", "col3"])

df.show()
# max_df=df.withColumn("maximum_no",F.greatest("col1","col2","col3"))
# min_of_max=max_df.select(F.min("maximum_no")).collect()[0][0]
# print("minimum of maximum by each row is:",min_of_max)
# max_df.show()
min_max_df=df.withColumn("row_min",F.least("col1","col2","col3")) \
    .withColumn("row_max",F.greatest("col1","col2","col3"))
result_df=min_max_df.withColumn("min_by_max",F.col("row_min")/F.col("row_max"))
result_df.show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|   2|   3|
|   4|   5|   6|
|   7|   8|   9|
|  10|  11|  12|
+----+----+----+

+----+----+----+-------+-------+------------------+
|col1|col2|col3|row_min|row_max|        min_by_max|
+----+----+----+-------+-------+------------------+
|   1|   2|   3|      1|      3|0.3333333333333333|
|   4|   5|   6|      4|      6|0.6666666666666666|
|   7|   8|   9|      7|      9|0.7777777777777778|
|  10|  11|  12|     10|     12|0.8333333333333334|
+----+----+----+-------+-------+------------------+



41. How to create a column that contains the penultimate value in each row?

In [50]:
data = [(10, 20, 30),
(40, 60, 50),
(80, 70, 90)]

df = spark.createDataFrame(data, ["Column1", "Column2", "Column3"])

df.show()

from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, IntegerType,DoubleType

# Define UDF to sort array in descending order
def sort_row_get_second(row):
    return sorted(row,reverse=True)[1]
sort_row_get_second_udf=udf(sort_row_get_second,IntegerType())
df_with_second=df.withColumn("penultimate",sort_row_get_second_udf(array("Column1", "Column2", "Column3")))
df_with_second.show()



+-------+-------+-------+
|Column1|Column2|Column3|
+-------+-------+-------+
|     10|     20|     30|
|     40|     60|     50|
|     80|     70|     90|
+-------+-------+-------+

+-------+-------+-------+-----------+
|Column1|Column2|Column3|penultimate|
+-------+-------+-------+-----------+
|     10|     20|     30|         20|
|     40|     60|     50|         50|
|     80|     70|     90|         80|
+-------+-------+-------+-----------+



42. How to normalize all columns in a dataframe?

In [51]:
# create a sample dataframe
data = [(1, 2, 3),
(2, 3, 4),
(3, 4, 5),
(4, 5, 6)]

df = spark.createDataFrame(data, ["Col1", "Col2", "Col3"])

df.show()

from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.functions import col

# define the list of columns to be normalized
input_cols = ["Col1", "Col2", "Col3"]

# initialize VectorAssembler with input and output column names
assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

# transform the data
df_assembled = assembler.transform(df)
df_assembled.show()
# initialize StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)

# fit and transform the data
scalerModel = scaler.fit(df_assembled)
df_normalized = scalerModel.transform(df_assembled)
df_normalized.show()


+----+----+----+
|Col1|Col2|Col3|
+----+----+----+
|   1|   2|   3|
|   2|   3|   4|
|   3|   4|   5|
|   4|   5|   6|
+----+----+----+

+----+----+----+-------------+
|Col1|Col2|Col3|     features|
+----+----+----+-------------+
|   1|   2|   3|[1.0,2.0,3.0]|
|   2|   3|   4|[2.0,3.0,4.0]|
|   3|   4|   5|[3.0,4.0,5.0]|
|   4|   5|   6|[4.0,5.0,6.0]|
+----+----+----+-------------+

+----+----+----+-------------+--------------------+
|Col1|Col2|Col3|     features|     scaled_features|
+----+----+----+-------------+--------------------+
|   1|   2|   3|[1.0,2.0,3.0]|[-1.1618950038622...|
|   2|   3|   4|[2.0,3.0,4.0]|[-0.3872983346207...|
|   3|   4|   5|[3.0,4.0,5.0]|[0.38729833462074...|
|   4|   5|   6|[4.0,5.0,6.0]|[1.16189500386222...|
+----+----+----+-------------+--------------------+



43. How to get the positions where values of two columns match?

In [52]:
# Create sample DataFrame
data = [("John", "John"), ("Lily", "Lucy"), ("Sam", "Sam"), ("Lucy", "Lily")]
df = spark.createDataFrame(data, ["Name1", "Name2"])

df.show()
filtered_df=df.withColumn("position",when (col("Name1")==col("Name2"),True).otherwise(False)
)

filtered_df.show()

+-----+-----+
|Name1|Name2|
+-----+-----+
| John| John|
| Lily| Lucy|
|  Sam|  Sam|
| Lucy| Lily|
+-----+-----+

+-----+-----+--------+
|Name1|Name2|position|
+-----+-----+--------+
| John| John|    true|
| Lily| Lucy|   false|
|  Sam|  Sam|    true|
| Lucy| Lily|   false|
+-----+-----+--------+



44. How to create lags and leads of a column by group in a dataframe?

In [53]:
# Create a sample DataFrame
data = [("2023-01-01", "Store1", 100),
("2023-01-02", "Store1", 150),
("2023-01-03", "Store1", 200),
("2023-01-04", "Store1", 250),
("2023-01-05", "Store1", 300),
("2023-01-01", "Store2", 50),
("2023-01-02", "Store2", 60),
("2023-01-03", "Store2", 80),
("2023-01-04", "Store2", 90),
("2023-01-05", "Store2", 120)]

df = spark.createDataFrame(data, ["Date", "Store", "Sales"])

df.show()
windowspec=Window.partitionBy("Store").orderBy("Date")

new_df=df.withColumn("lag",lag("Sales",2).over(windowspec))\
    .withColumn("lead",lead("Sales",2).over(windowspec))
new_df.show()

+----------+------+-----+
|      Date| Store|Sales|
+----------+------+-----+
|2023-01-01|Store1|  100|
|2023-01-02|Store1|  150|
|2023-01-03|Store1|  200|
|2023-01-04|Store1|  250|
|2023-01-05|Store1|  300|
|2023-01-01|Store2|   50|
|2023-01-02|Store2|   60|
|2023-01-03|Store2|   80|
|2023-01-04|Store2|   90|
|2023-01-05|Store2|  120|
+----------+------+-----+

+----------+------+-----+----+----+
|      Date| Store|Sales| lag|lead|
+----------+------+-----+----+----+
|2023-01-01|Store1|  100|NULL| 200|
|2023-01-02|Store1|  150|NULL| 250|
|2023-01-03|Store1|  200| 100| 300|
|2023-01-04|Store1|  250| 150|NULL|
|2023-01-05|Store1|  300| 200|NULL|
|2023-01-01|Store2|   50|NULL|  80|
|2023-01-02|Store2|   60|NULL|  90|
|2023-01-03|Store2|   80|  50| 120|
|2023-01-04|Store2|   90|  60|NULL|
|2023-01-05|Store2|  120|  80|NULL|
+----------+------+-----+----+----+



45. How to get the frequency of unique values in the entire dataframe?

In [54]:
data = [(1, 2, 3),
(2, 3, 4),
(1, 2, 3),
(4, 5, 6),
(2, 3, 4)]
df = spark.createDataFrame(data, ["Column1", "Column2", "Column3"])

# # Print DataFrame
# df.show()
# col1_count=df.groupBy("Column1").count()
# col2_count=df.groupBy("Column2").count()
# col3_count=df.groupBy("Column3").count()
# col1_count.show()
# col2_count.show()
# col3_count.show()

#or

combined_df=df.selectExpr("Column1 as Value").union(df.selectExpr("Column2 as value")).union(df.selectExpr("Column3 as Value"))
value_count=combined_df.groupBy("Value").count()
value_count.show()

+-----+-----+
|Value|count|
+-----+-----+
|    1|    2|
|    2|    4|
|    4|    3|
|    5|    1|
|    3|    4|
|    6|    1|
+-----+-----+



46. How to replace both the diagonals of dataframe with 0?

In [55]:
# Create a numeric DataFrame
data = [(1, 2, 3, 4),
(2, 3, 4, 5),
(1, 2, 3, 4),
(4, 5, 6, 7)]

df = spark.createDataFrame(data, ["col_1", "col_2", "col_3", "col_4"])

# Print DataFrame
df.show()
n = len(data)

# Replace both diagonal elements with 0
for i in range(n):
    df = df.withColumn("Col_" + str(i + 1), 
                       F.when(F.row_number().over(Window.orderBy(F.lit(1))) - 1 == i, 0)  # Main diagonal condition
                       .when(F.row_number().over(Window.orderBy(F.lit(1))) - 1 == n - 1 - i, 0) # Anti-diagonal condition
                       .otherwise(F.col("Col_" + str(i + 1))))

# Show the updated DataFrame
print("DataFrame after replacing both diagonal elements with 0:")
df.show()

+-----+-----+-----+-----+
|col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
|    1|    2|    3|    4|
|    2|    3|    4|    5|
|    1|    2|    3|    4|
|    4|    5|    6|    7|
+-----+-----+-----+-----+

DataFrame after replacing both diagonal elements with 0:


24/09/25 11:45:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+-----+-----+-----+
|Col_1|Col_2|Col_3|Col_4|
+-----+-----+-----+-----+
|    0|    2|    3|    0|
|    2|    0|    0|    5|
|    1|    0|    0|    4|
|    0|    5|    6|    0|
+-----+-----+-----+-----+



47. How to reverse the rows of a dataframe?

In [56]:
data = [(1, 2, 3, 4),
(2, 3, 4, 5),
(3, 4, 5, 6),
(4, 5, 6, 7)]
# by directly affecting the input 
# data=data[::-1]

df = spark.createDataFrame(data, ["col_1", "col_2", "col_3", "col_4"])

# Print DataFrame
# data=data[::-1]

# reversing the dataframe without changing the input
w=Window.orderBy(monotonically_increasing_id())
df=df.withColumn("id",row_number().over(w)-1)
df2=df.orderBy("id",ascending=False).drop("id")
df2.show()

+-----+-----+-----+-----+
|col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
|    4|    5|    6|    7|
|    3|    4|    5|    6|
|    2|    3|    4|    5|
|    1|    2|    3|    4|
+-----+-----+-----+-----+



24/09/25 11:45:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/25 11:45:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


48. How to create one-hot encodings of a categorical variable (dummy variables)?


In [57]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

In [58]:
data = [("A", 10),("A", 20),("B", 30),("B", 20),("B", 30),("C", 40),("C", 10),("D", 10)]
columns = ["Categories", "Value"]

df = spark.createDataFrame(data, columns)
#before performing one Hot Encoding  perform Sting Intexer so that the string will be assigned values.
indexer=StringIndexer(inputCol="Categories",outputCol="category_numeric")
indexer_fitted=indexer.fit(df)
df_indexed=indexer_fitted.transform(df)
df_indexed.show()
encoder=OneHotEncoder(inputCols=["category_numeric"],outputCols=["category_encoded"])
df_onehot=encoder.fit(df_indexed).transform(df_indexed)
df_onehot.show()

+----------+-----+----------------+
|Categories|Value|category_numeric|
+----------+-----+----------------+
|         A|   10|             1.0|
|         A|   20|             1.0|
|         B|   30|             0.0|
|         B|   20|             0.0|
|         B|   30|             0.0|
|         C|   40|             2.0|
|         C|   10|             2.0|
|         D|   10|             3.0|
+----------+-----+----------------+

+----------+-----+----------------+----------------+
|Categories|Value|category_numeric|category_encoded|
+----------+-----+----------------+----------------+
|         A|   10|             1.0|   (3,[1],[1.0])|
|         A|   20|             1.0|   (3,[1],[1.0])|
|         B|   30|             0.0|   (3,[0],[1.0])|
|         B|   20|             0.0|   (3,[0],[1.0])|
|         B|   30|             0.0|   (3,[0],[1.0])|
|         C|   40|             2.0|   (3,[2],[1.0])|
|         C|   10|             2.0|   (3,[2],[1.0])|
|         D|   10|             3.0|  

49. How to Pivot the dataframe (converting rows into columns) ?

In [59]:
# Sample data
data = [
(2021, 1, "US", 5000),
(2021, 1, "EU", 4000),
(2021, 2, "US", 5500),
(2021, 2, "EU", 4500),
(2021, 3, "US", 6000),
(2021, 3, "EU", 5000),
(2021, 4, "US", 7000),
(2021, 4, "EU", 6000),
]

# Create DataFrame
columns = ["year", "quarter", "region", "revenue"]
df = spark.createDataFrame(data, columns)
df.show()
pivot_df=df.groupBy("year","quarter").pivot("region").sum("revenue")
pivot_df.show()

+----+-------+------+-------+
|year|quarter|region|revenue|
+----+-------+------+-------+
|2021|      1|    US|   5000|
|2021|      1|    EU|   4000|
|2021|      2|    US|   5500|
|2021|      2|    EU|   4500|
|2021|      3|    US|   6000|
|2021|      3|    EU|   5000|
|2021|      4|    US|   7000|
|2021|      4|    EU|   6000|
+----+-------+------+-------+

+----+-------+----+----+
|year|quarter|  EU|  US|
+----+-------+----+----+
|2021|      2|4500|5500|
|2021|      1|4000|5000|
|2021|      4|6000|7000|
|2021|      3|5000|6000|
+----+-------+----+----+



50. How to UnPivot the dataframe (converting columns into rows) ?

In [60]:
unpivot_expr="stack(2,'US','US','EU',EU') as (region,revenue)"
unpivot_df=pivot_df.select("year","quarter",F.expr(unpivot_expr)).where("revenue is not null")
unpivot_df.show()

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near ''': extra input '''.(line 1, pos 25)

== SQL ==
stack(2,'US','US','EU',EU') as (region,revenue)
-------------------------^^^


In [ ]:
# Sample data
data = [(2021, 2, 4500, 5500),
(2021, 1, 4000, 5000),
(2021, 3, 5000, 6000),
(2021, 4, 6000, 7000)]

# Create DataFrame
columns = ["year", "quarter", "EU", "US"]
df = spark.createDataFrame(data, columns)

df.show()

51. How to impute missing values with Zero?

In [125]:
# Suppose df is your DataFrame
df = spark.createDataFrame([(1, None), (None, 2), (3, 4), (5, None)], ["a", "b"])
imputed_df=df.fillna(0)
imputed_df.show()

+---+---+
|  a|  b|
+---+---+
|  1|  0|
|  0|  2|
|  3|  4|
|  5|  0|
+---+---+



52. How to identify continuous variables in a dataframe and create a list of those column names?

In [129]:
from pyspark.sql.types import IntegerType, StringType, NumericType
from pyspark.sql.functions import approxCountDistinct

In [146]:
url = "https://raw.githubusercontent.com/selva86/datasets/master/Churn_Modelling_m.csv"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("Churn_Modelling_m.csv"), header=True, inferSchema=True)

#df = spark.read.csv("C:/Users/RajeshVaddi/Documents/MLPlus/DataSets/Churn_Modelling_m.csv", header=True, inferSchema=True)
# df.show()


def detect_continuous_variables(df, distinct_threshold):

    continuous_columns = []
    for column in df.columns:
        dtype = df.schema[column].dataType
    if isinstance(dtype, (IntegerType, NumericType)):
        distinct_count = df.select(approxCountDistinct(column)).collect()[0][0]
    if distinct_count > distinct_threshold:
        continuous_columns.append(column)
    return continuous_columns

continuous_variables = detect_continuous_variables(df, 10)
print(continuous_variables)

24/09/24 18:12:29 WARN SparkContext: The path https://raw.githubusercontent.com/selva86/datasets/master/Churn_Modelling_m.csv has been added already. Overwriting of added paths is not supported in the current version.


[]


/home/ai/.local/lib/python3.10/site-packages/pyspark/sql/functions.py:3796: FutureWarning: Deprecated in 2.1, use approx_count_distinct instead.
  warnings.warn("Deprecated in 2.1, use approx_count_distinct instead.", FutureWarning)


53. How to calculate Mode of a PySpark DataFrame column?

In [133]:
# Create a sample DataFrame
data = [(1, 2, 3), (2, 2, 3), (2, 2, 4), (1, 2, 3), (1, 1, 3)]
columns = ["col1", "col2", "col3"]

df = spark.createDataFrame(data, columns)

df.show()

from pyspark.sql.functions import col

df_grouped = df.groupBy('col2').count()
mode_df = df_grouped.orderBy(col('count').desc()).limit(1)

mode_df.show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|   2|   3|
|   2|   2|   3|
|   2|   2|   4|
|   1|   2|   3|
|   1|   1|   3|
+----+----+----+

+----+-----+
|col2|count|
+----+-----+
|   2|    4|
+----+-----+



54. How to find installed location of Apache Spark and PySpark?

In [134]:
pip install findspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [135]:
import findspark
findspark.init()

print(findspark.find())

import os
import pyspark

print(os.path.dirname(pyspark.__file__))

/opt/spark
/home/ai/.local/lib/python3.10/site-packages/pyspark


55 How to convert a column to lower case using UDF?

In [145]:
# Create a DataFrame to test
data = [('John Doe', 'NEW YORK'),
('Jane Doe', 'LOS ANGELES'),
('Mike Johnson', 'CHICAGO'),
('Sara Smith', 'SAN FRANCISCO')]

df = spark.createDataFrame(data, ['Name', 'City'])

df.show()
def to_lower(s):
    if s is not None:
        return s.lower()

# Convert your Python function to a Spark UDF
udf_to_lower = udf(to_lower, StringType())

# Apply your UDF to the DataFrame
df = df.withColumn('City_lower', udf_to_lower(df['City']))

# Show the DataFrame
df.show()

+------------+-------------+
|        Name|         City|
+------------+-------------+
|    John Doe|     NEW YORK|
|    Jane Doe|  LOS ANGELES|
|Mike Johnson|      CHICAGO|
|  Sara Smith|SAN FRANCISCO|
+------------+-------------+

+------------+-------------+-------------+
|        Name|         City|   City_lower|
+------------+-------------+-------------+
|    John Doe|     NEW YORK|     new york|
|    Jane Doe|  LOS ANGELES|  los angeles|
|Mike Johnson|      CHICAGO|      chicago|
|  Sara Smith|SAN FRANCISCO|san francisco|
+------------+-------------+-------------+



57. How to View PySpark Cluster Details?

In [139]:
print(spark.sparkContext.uiWebUrl)

http://192.168.1.164:4040


58. How to View PySpark Cluster Configuration Details?


In [141]:
for k,v in spark.sparkContext.getConf().getAll():
    print(f"{k} : {v}")

spark.driver.extraJavaOptions : -Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false
spark.app.name : SparkAssessment.com
spark.driver.port : 34343
spark.executor.id : driver
spark.sql.warehouse.di

59. How to restrict the PySpark to use the number of cores in the system?

In [142]:
from pyspark import SparkConf, SparkContext

conf = SparkConf()
conf.set("spark.executor.cores", "2") # set the number of cores you want here

60. How to cache PySpark DataFrame or objects and delete cache?


In [143]:
df.cache()
df.unpersist()

DataFrame[Name: string, City: string, City_lower: string]

61. How to Divide a PySpark DataFrame randomly in a given ratio (0.8, 0.2)?

In [149]:
data = [(1, "John"), (2, "Alice"), (3, "Bob"), (4, "Charlie"), (5, "David")]
df = spark.createDataFrame(data, ["id", "name"])

In [152]:
splits= df.randomSplit([0.8, 0.2], seed=42)
subset1 = splits[0]
subset2 = splits[1]

# Show the subsets
subset1.show()
subset2.show()

+---+-------+
| id|   name|
+---+-------+
|  1|   John|
|  2|  Alice|
|  4|Charlie|
|  5|  David|
+---+-------+

+---+----+
| id|name|
+---+----+
|  3| Bob|
+---+----+



62. How to build logistic regression in PySpark?

In [154]:
# Create a sample dataframe
data = spark.createDataFrame([
(0, 1.0, -1.0),
(1, 2.0, 1.0),
(1, 3.0, -2.0),
(0, 4.0, 1.0),
(1, 5.0, -3.0),
(0, 6.0, 2.0),
(1, 7.0, -1.0),
(0, 8.0, 3.0),
(1, 9.0, -2.0),
(0, 10.0, 2.0),
(1, 11.0, -3.0),
(0, 12.0, 1.0),
(1, 13.0, -1.0),
(0, 14.0, 2.0),
(1, 15.0, -2.0),
(0, 16.0, 3.0),
(1, 17.0, -3.0),
(0, 18.0, 1.0),
(1, 19.0, -1.0),
(0, 20.0, 2.0)
], ["label", "feat1", "feat2"])
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

# convert the feature columns into a single vector column using VectorAssembler
vecAssembler = VectorAssembler(inputCols=['feat1', 'feat2'], outputCol="features")
data = vecAssembler.transform(data)

# fit the logistic regression model
lr = LogisticRegression(featuresCol='features', labelCol='label')
lr_model = lr.fit(data)

# look at the coefficients and intercept of the logistic regression model
print(f"Coefficients: {str(lr_model.coefficients)}")
print(f"Intercept: {str(lr_model.intercept)}")



24/09/24 18:15:57 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


Coefficients: [0.02027774047578675,-1.6129609400223657]
Intercept: -0.22092927518295366


63. How to convert the categorical string data into numerical data or index?

In [157]:
# Create a sample DataFrame
data = [('cat',), ('dog',), ('mouse',), ('fish',), ('dog',), ('cat',), ('mouse',)]
df = spark.createDataFrame(data, ["animal"])

df.show()

# Initialize a StringIndexer
indexer = StringIndexer(inputCol='animal', outputCol='animalIndex')

# Fit the indexer to the DataFrame and transform the data
indexed = indexer.fit(df).transform(df)
indexed.show()

+------+
|animal|
+------+
|   cat|
|   dog|
| mouse|
|  fish|
|   dog|
|   cat|
| mouse|
+------+

+------+-----------+
|animal|animalIndex|
+------+-----------+
|   cat|        0.0|
|   dog|        1.0|
| mouse|        2.0|
|  fish|        3.0|
|   dog|        1.0|
|   cat|        0.0|
| mouse|        2.0|
+------+-----------+



64. How to calculate Correlation of two variables in a DataFrame?

In [156]:
# Create a sample dataframe
data = [Row(feature1=5, feature2=10, feature3=25),
Row(feature1=6, feature2=15, feature3=35),
Row(feature1=7, feature2=25, feature3=30),
Row(feature1=8, feature2=20, feature3=60),
Row(feature1=9, feature2=30, feature3=70)]
df = spark.createDataFrame(data)

df.show()
correlation=df.corr("feature1","feature2")
print(correlation)

+--------+--------+--------+
|feature1|feature2|feature3|
+--------+--------+--------+
|       5|      10|      25|
|       6|      15|      35|
|       7|      25|      30|
|       8|      20|      60|
|       9|      30|      70|
+--------+--------+--------+

0.9


65. How to calculate Correlation Matrix?

In [159]:
# Create a sample dataframe
data = [Row(feature1=5, feature2=10, feature3=25),
Row(feature1=6, feature2=15, feature3=35),
Row(feature1=7, feature2=25, feature3=30),
Row(feature1=8, feature2=20, feature3=60),
Row(feature1=9, feature2=30, feature3=70)]
df = spark.createDataFrame(data)

df.show()
#  Calculate Correlation Using Using MLlib
from pyspark.ml.stat import Correlation

# Assemble feature vector
# Define the feature and label columns & Assemble the feature vector
vector_assembler = VectorAssembler(inputCols=["feature1", "feature2", "feature3"], outputCol="features")
data_vector = vector_assembler.transform(df).select("features")

# Calculate correlation
correlation_matrix = Correlation.corr(data_vector, "features").head()[0]

print(correlation_matrix)


+--------+--------+--------+
|feature1|feature2|feature3|
+--------+--------+--------+
|       5|      10|      25|
|       6|      15|      35|
|       7|      25|      30|
|       8|      20|      60|
|       9|      30|      70|
+--------+--------+--------+

DenseMatrix([[1.        , 0.9       , 0.91779992],
             [0.9       , 1.        , 0.67837385],
             [0.91779992, 0.67837385, 1.        ]])


66. How to calculate VIF (Variance Inflation Factor ) for set of variables in a DataFrame?

In [160]:
# Create a sample dataframe
data = [Row(feature1=5, feature2=10, feature3=25),
Row(feature1=6, feature2=15, feature3=35),
Row(feature1=7, feature2=25, feature3=30),
Row(feature1=8, feature2=20, feature3=60),
Row(feature1=9, feature2=30, feature3=70)]
df = spark.createDataFrame(data)

df.show()

+--------+--------+--------+
|feature1|feature2|feature3|
+--------+--------+--------+
|       5|      10|      25|
|       6|      15|      35|
|       7|      25|      30|
|       8|      20|      60|
|       9|      30|      70|
+--------+--------+--------+



In [161]:
from pyspark.sql import SparkSession, Row
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

def calculate_vif(data, features):
    vif_dict = {}

    for feature in features:
        non_feature_cols = [col for col in features if col != feature]
    assembler = VectorAssembler(inputCols=non_feature_cols, outputCol="features")
    lr = LinearRegression(featuresCol='features', labelCol=feature)

    model = lr.fit(assembler.transform(data))
    vif = 1 / (1 - model.summary.r2)

    vif_dict[feature] = vif

    return vif_dict

features = ['feature1', 'feature2', 'feature3']
vif_values = calculate_vif(df, features)

for feature, vif in vif_values.items():
    print(f'VIF for {feature}: {vif}')

24/09/24 18:21:44 WARN Instrumentation: [be5076c3] regParam is zero, which might cause numerical instability and overfitting.


VIF for feature3: 23.30468749999992


67. How to perform Chi-Square test?

In [163]:
# Create a sample dataframe
data = [(1, 0, 0, 1, 1),
(2, 0, 1, 0, 0),
(3, 1, 0, 0, 0),
(4, 0, 0, 1, 1),
(5, 0, 1, 1, 0)]

df = spark.createDataFrame(data, ["id", "feature1", "feature2", "feature3", "label"])

df.show()
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["feature1", "feature2", "feature3"], outputCol="features")
df = assembler.transform(df)

from pyspark.ml.stat import ChiSquareTest

r = ChiSquareTest.test(df, "features", "label").head()
print("pValues: " + str(r.pValues))
print("degreesOfFreedom: " + str(r.degreesOfFreedom))
print("statistics: " + str(r.statistics))

+---+--------+--------+--------+-----+
| id|feature1|feature2|feature3|label|
+---+--------+--------+--------+-----+
|  1|       0|       0|       1|    1|
|  2|       0|       1|       0|    0|
|  3|       1|       0|       0|    0|
|  4|       0|       0|       1|    1|
|  5|       0|       1|       1|    0|
+---+--------+--------+--------+-----+

pValues: [0.36131042852617856,0.13603712811414348,0.1360371281141436]
degreesOfFreedom: [1, 1, 1]
statistics: [0.8333333333333335,2.2222222222222228,2.2222222222222223]


68. How to calculate the Standard Deviation?

In [164]:
# Sample data
data = [("James", "Sales", 3000),
("Michael", "Sales", 4600),
("Robert", "Sales", 4100),
("Maria", "Finance", 3000),
("James", "Sales", 3000),
("Scott", "Finance", 3300),
("Jen", "Finance", 3900),
("Jeff", "Marketing", 3000),
("Kumar", "Marketing", 2000),
("Saif", "Sales", 4100)]

# Create DataFrame
df = spark.createDataFrame(data, ["Employee", "Department", "Salary"])

df.show()
from pyspark.sql.functions import stddev

salary_stddev = df.select(stddev("Salary").alias("stddev"))

salary_stddev.show()

+--------+----------+------+
|Employee|Department|Salary|
+--------+----------+------+
|   James|     Sales|  3000|
| Michael|     Sales|  4600|
|  Robert|     Sales|  4100|
|   Maria|   Finance|  3000|
|   James|     Sales|  3000|
|   Scott|   Finance|  3300|
|     Jen|   Finance|  3900|
|    Jeff| Marketing|  3000|
|   Kumar| Marketing|  2000|
|    Saif|     Sales|  4100|
+--------+----------+------+

+-----------------+
|           stddev|
+-----------------+
|765.9416862050705|
+-----------------+



69. How to calculate missing value percentage in each column?

In [166]:
# Create a sample dataframe
data = [("John", "Doe", None),
(None, "Smith", "New York"),
("Mike", "Smith", None),
("Anna", "Smith", "Boston"),
(None, None, None)]

df = spark.createDataFrame(data, ["FirstName", "LastName", "City"])

df.show()

# Calculate the total number of rows in the dataframe
total_rows = df.count()

# For each column calculate the number of null values and then calculate the percentage
for column in df.columns:
    null_values = df.filter(df[column].isNull()).count()
    missing_percentage = (null_values / total_rows) * 100
    print(f"Missing values in {column}: {missing_percentage}%")

+---------+--------+--------+
|FirstName|LastName|    City|
+---------+--------+--------+
|     John|     Doe|    NULL|
|     NULL|   Smith|New York|
|     Mike|   Smith|    NULL|
|     Anna|   Smith|  Boston|
|     NULL|    NULL|    NULL|
+---------+--------+--------+

Missing values in FirstName: 40.0%
Missing values in LastName: 20.0%
Missing values in City: 60.0%


70. How to get the names of DataFrame objects that have been created in an environment?

In [168]:
dataframe_names = [name for name, obj in globals().items() if isinstance(obj, pyspark.sql.DataFrame)]

for name in dataframe_names:
    print(name)

_
df
df_with_index
df_index
name_count
result_df
df_buck
mul_3
df_A
df_B
res_df
squared_error
Describe
df_with_diff
df_with_diff_of_diff
df_with_dates
df_final
char_df
c_count
saturdays
summary_stat
null_count
imputed_df
df_percentage
df_sum
filtered_df
min_max_df
df_with_second
df_assembled
df_normalized
new_df
col1_count
col2_count
col3_count
combined_df
value_count
df2
df_indexed
df_onehot
pivot_df
df_grouped
mode_df
_143
train_data
test_data
subset1
subset2
indexed
data_vector
salary_stddev
